In [ ]:
!pip install adabelief-pytorch==0.1.0

In [ ]:
import time
import torch
import timeit
import numpy as np
import torch.nn as nn
import torch.optim as optim
from adabelief_pytorch import AdaBelief
from torch.utils import data
import matplotlib.pyplot as plt
import torch.nn.functional as F
from collections import namedtuple
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

%matplotlib inline
np.random.seed(2018)

In [ ]:
train = np.load('C:/Users/malji/Google Drive/Colab Notebooks/Hw3/ntrain.npy',allow_pickle=True)
train_labels = np.load('C:/Users/malji/Google Drive/Colab Notebooks/Hw3/ntrain_labels.npy',allow_pickle=True)
val = np.load('C:/Users/malji/Google Drive/Colab Notebooks/Hw3/nval.npy',allow_pickle=True)
val_labels = np.load('C:/Users/malji/Google Drive/Colab Notebooks/Hw3/nval_labels.npy',allow_pickle=True)
ntest = np.load('C:/Users/malji/Google Drive/Colab Notebooks/Hw3/ntest.npy',allow_pickle=True)
phones = np.loadtxt("C:/Users/malji/Google Drive/Colab Notebooks/Hw3/phones.txt", dtype=str)

In [ ]:
class PhonesDataset(Dataset):
  def __init__(self, x,y,k):             
    self.x = x
    self.y = y
    self.k = k
    self.inputs = []
    self.lab = np.array([])
    self.idx = []
    self._init_dataset()
      
  def __len__(self):
    return len(self.lab)

  def __getitem__(self,index):
    i = self.idx[index]
    return torch.from_numpy(np.concatenate(self.inputs[i-self.k:i+self.k+1],axis=0)).float(),torch.tensor(self.lab[index]).long()
  
  def _init_dataset(self):
    idx=0
    for i in range(len(self.x)):
      start_time = time.time()#######
      s = len(self.x[i])
      x=np.pad(self.x[i],((self.k, self.k), (0, 0)), 'constant', constant_values=0)

      idx += 2*self.k
      for j in range(s):           
        self.idx+=[idx-self.k]
        idx += 1

      self.inputs+=list(x)  
      self.lab = np.append(self.lab,self.y[i])
      end_time = time.time()#######  
    return np.array(self.inputs), self.lab

In [ ]:
class TestDataset(Dataset):
  def __init__(self, x,k):       
    self.x = x
    self.k = k
    self.inputs = []
    self.idx = []
    self._init_dataset()
      
  def __len__(self):
    return len(self.idx)

  def __getitem__(self,index):
    idx = self.idx[index]
    return torch.from_numpy(np.concatenate(self.inputs[idx-self.k:idx+self.k+1],axis=0)).float()
  
  def _init_dataset(self):
    idx=0
    for i in range(len(self.x)):
      start_time = time.time()#######
      s = len(self.x[i])
      x=np.pad(self.x[i],((self.k, self.k), (0, 0)), 'constant', constant_values=0)

      idx += 2*self.k
      for j in range(s):           
        self.idx+=[idx-self.k]
        idx += 1

      self.inputs+=list(x)  
      end_time = time.time()#######
      if i==1000:##########
        print('Time: ',end_time - start_time, 's') #######    
    return np.array(self.inputs)

In [ ]:
# def save_data(loader):
#     print('saving data...')
    
#     for i, (x, y) in enumerate(loader):
        
#         x=x.view(-1, 13).numpy()
#         y = y.numpy()
#         if i==0:
#             data = np.array(x)
#             label = np.array(y)
#         else:     
#             data = np.concatenate((data, x))
#             label = np.concatenate((label, y))
#     return data, label


In [ ]:
# #saving datas
# start_time = time.time()
# print("saving data...")
# data, labels = save_data(train_loader)
# np.save('C:/Users/malji/Google Drive/Colab Notebooks/Hw3/train_data_new.npy', data)
# np.save('C:/Users/malji/Google Drive/Colab Notebooks/Hw3/train_labels_new.npy', labels)
# print("train data savied in:")
# print("--- %s seconds ---" % (time.time() - start_time))
# print("\n===================================\n")


In [ ]:
cuda = torch.cuda.is_available()
num_workers = 0 #8 if cuda else 0 
    
# Training
start_time = time.time()
train_dataset = PhonesDataset(train, train_labels,50)
train_loader_args = dict(shuffle=True, batch_size=256, num_workers=num_workers, pin_memory=True)
train_loader = data.DataLoader(train_dataset, **train_loader_args)
print("taken time: %s seconds ---" % (time.time() - start_time))

taken time: 812.9333379268646 seconds ---


In [ ]:
# Validation
start_time = time.time()
num_workers = 0 #8 
val_dataset = PhonesDataset(val, val_labels,50)
val_loader_args = dict(shuffle=False, batch_size=256, num_workers=num_workers, pin_memory=True)
val_loader = data.DataLoader(val_dataset, **val_loader_args)
print("taken time: %s seconds ---" % (time.time() - start_time))

taken time: 7.3186492919921875 seconds ---


In [ ]:
# Testing
start_time = time.time()
test_dataset = TestDataset(ntest,50)
test_loader_args = dict(shuffle=False, batch_size=1, num_workers=num_workers, pin_memory=True)
test_loader = data.DataLoader(test_dataset, **test_loader_args)
print("taken time: %s seconds ---" % (time.time() - start_time))

Time:  0.0 s
taken time: 2.0988807678222656 seconds ---


In [ ]:
def init_xavier(m):
  if type(m) == nn.Linear:
    fan_in = m.weight.size()[1]
    fan_out = m.weight.size()[0]
    std = np.sqrt(1.0/(fan_in + fan_out))
    m.weight.data.normal_(0,std)


In [ ]:
def init_hey(m):
  if type(m) == nn.Linear:
    fan_in = m.weight.size()[1]
    fan_out = m.weight.size()[0]
    std = np.sqrt(2.0/(fan_in + fan_out))
    m.weight.data.normal_(0,std)


In [ ]:
class PhonesModel(nn.Module):
  # try changing 32 to 128
  def __init__(self):
    super(PhonesModel, self).__init__()
    self.fc1 = nn.Linear(1313, 4096 )
    self.bnorm1 = nn.BatchNorm1d(4096 )
    self.dp1 = nn.Dropout(p=0.1)
    self.fc2 = nn.Linear(4096, 2048 )
    self.bnorm2 = nn.BatchNorm1d(2048 )
    self.dp2 = nn.Dropout(p=0.1)
    self.fc3 = nn.Linear(2048, 1024 )
    self.bnorm3 = nn.BatchNorm1d(1024 )
    self.dp3 = nn.Dropout(p=0.1)
    self.fc4 = nn.Linear(1024 , 1024 )
    self.bnorm4 = nn.BatchNorm1d(1024 )
    self.dp4 = nn.Dropout(p=0.1)
    self.fc5 = nn.Linear(1024, 512 )
    self.bnorm5 = nn.BatchNorm1d(512 )
    self.dp5 = nn.Dropout(p=0.1)
    self.fc6 = nn.Linear(512, 256 )
    self.bnorm6 = nn.BatchNorm1d(256 )
    self.dp6 = nn.Dropout(p=0.1)
    self.fc7 = nn.Linear(256 , 346)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.dp1(self.bnorm1(x))
    x = F.relu(self.fc2(x))
    x = self.dp2(self.bnorm2(x))
    x = F.relu(self.fc3(x))
    x = self.dp3(self.bnorm3(x))
    x = F.relu(self.fc4(x))
    x = self.dp4(self.bnorm4(x))
    x = F.relu(self.fc5(x))
    x = self.dp5(self.bnorm5(x))
    x = F.relu(self.fc6(x))
    x = self.dp6(self.bnorm6(x))
    x = F.log_softmax(self.fc7(x))
    return x

In [ ]:
model = PhonesModel()
model.apply(init_xavier)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters())
optimizer = AdaBelief(model.parameters(), lr=1e-3, eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
#scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2, verbose=True)
device = torch.device("cuda" if cuda else "cpu")
model.to(device)
print(model)


Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
Current version (0.1.0)  1e-16  True               True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Weight decoupling enabled in AdaBelief
PhonesModel(
  (fc1): Linear(in_features=1313, out_features=4096, bias=True)
  (bnorm1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp1): Dropout(p=0.1, inplace=False)
  (fc2): Linear(in_features=4096, out_features=2048, bias=True)
  (bnorm2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp2): Dropout(p=0.1, inplace=False)
  (fc3): Linear(in_features=2048, out_features=1024, bias=True)
  (bnorm3): BatchNorm1

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer):
  model.train()

  running_loss = 0.0
  total_predictions = 0.0
  correct_predictions = 0.0
  model.to(device)
  
  start_time = time.time()
  
  # Print Learning Rate
  
  for batch_idx, (data, target) in enumerate(train_loader):   
    optimizer.zero_grad()   # .backward() accumulates gradients
    data = data.to(device)
    target = target.to(device) # all data & model on same device

    outputs = model(data)
    _, predicted = torch.max(outputs.data, 1)
    
    total_predictions += target.size(0)
    correct_predictions += (predicted == target).sum().item()
    
    loss = criterion(outputs, target)
    running_loss += loss.item()

    loss.backward()
    optimizer.step()
  scheduler.step()
  end_time = time.time()
  
  running_loss /= len(train_loader)
  acc = (correct_predictions/total_predictions)*100.0
  print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')
  print('Training Accuracy: ', acc, '%')
  return running_loss,acc


In [ ]:
def val_model(model, val_loader, criterion):
  with torch.no_grad():
    model.eval()
    model.to(device)

    running_loss = 0.0
    total_predictions = 0.0
    correct_predictions = 0.0

    for batch_idx, (data, target) in enumerate(val_loader):   
      data = data.to(device)
      target = target.to(device)

      outputs = model(data)

      _, predicted = torch.max(outputs.data, 1)
      total_predictions += target.size(0)
      correct_predictions += (predicted == target).sum().item()

      loss = criterion(outputs, target).detach()
      running_loss += loss.item()


    running_loss /= len(val_loader)
    acc = (correct_predictions/total_predictions)*100.0
    print('Validation Loss: ', running_loss)
    print('Validation Accuracy: ', acc, '%')
    return running_loss, acc


In [ ]:
def test_model(model, test_loader):
  with torch.no_grad():
    model.eval()
    pred = []

    for batch_idx, (data) in enumerate(test_loader):   
      data = data.to(device)
      outputs = model(data)

      _, predicted = torch.max(outputs.data, 1)
      pred.append(predicted.cpu().numpy()[0])

    return np.array(pred)


In [ ]:
n_epochs = 5
Train_acc = []
Train_loss = []
Val_loss = []
Val_acc = []
# loading our existing model
load_path_model = 'C:/Users/malji/Google Drive/Colab Notebooks/trained_model/trained_model_newXYZ.pt'
model.load_state_dict(torch.load(load_path_model))

for i in range(n_epochs):
  print('Epoch: ',i+1)
  print('LR: ', scheduler.get_lr())
  train_loss,acc = train_epoch(model, train_loader, criterion, optimizer)
  test_loss, test_acc = val_model(model, val_loader, criterion)
  Train_loss.append(train_loss)
  Train_acc.append(acc)
  Val_loss.append(test_loss)
  Val_acc.append(test_acc)
  print('='*20)
  #scheduler.step(test_acc)
  torch.save(model.state_dict(), 'C:/Users/malji/Google Drive/Colab Notebooks/trained_model/trained_model_newXYZ.pt')


Epoch:  1
LR:  [0.000125]


<ipython-input-13-38b4ca83bf3d>:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(self.fc7(x))


Training Loss:  0.05980727315243796 Time:  1692.618938922882 s
Training Accuracy:  98.0459553006551 %
Validation Loss:  3.1243579111191457
Validation Accuracy:  56.57918773977042 %
Epoch:  2
LR:  [0.00025]
Training Loss:  0.055534576328970416 Time:  1684.292457818985 s
Training Accuracy:  98.18641445752517 %
Validation Loss:  3.175539022880743
Validation Accuracy:  56.577811366838425 %
Epoch:  3
LR:  [0.00025]
Training Loss:  0.05339658571299918 Time:  1680.5725538730621 s
Training Accuracy:  98.2547226048038 %
Validation Loss:  3.228102258306944
Validation Accuracy:  56.578311866086416 %
Epoch:  4
LR:  [0.00025]
Training Loss:  0.05177729221489825 Time:  1680.6305301189423 s
Training Accuracy:  98.30889511317842 %
Validation Loss:  3.232001688703656
Validation Accuracy:  56.52619738188843 %
Epoch:  5
LR:  [0.00025]
Training Loss:  0.050491574157773694 Time:  1684.9339249134064 s
Training Accuracy:  98.34499177911025 %
Validation Loss:  3.232496380753467
Validation Accuracy:  56.554475